<h1 align="center"> Dream IPL team Data Scraping </h1>

**OBJECTIVE**

This project is about determining the best 11 players that can maximize the chances of winning the tournament based on analysis of data from the T20 world cup 2022/23 held in November 2022. BeautifulSoup library in Python was used to scrap the data from ESPN website. Data was further cleaned and preprocessed using various libraries in python and finally an interactive dashboard was built using PowerBI. The dashboard helps in 360 degree analysis of performance, potential and recent form of players to help shape the dream team driven by data.

In [ ]:
#libraires required
import pandas as pd
import json
from bs4 import BeautifulSoup # this module helps in web scrapping.
import requests  # this module helps us to download a web page

**1. Scraping Match Summary**

In [15]:
url='https://stats.espncricinfo.com/ci/engine/records/team/match_results.html?id=14450;type=tournament'

In [17]:
df_match=pd.read_html(url)[0]
df_match.head()


,Team 1,Team 2,Winner,Margin,Ground,Match Date,Scorecard
0,Namibia,Sri Lanka,Namibia,55 runs,Geelong,"Oct 16, 2022",T20I # 1823
1,Netherlands,U.A.E.,Netherlands,3 wickets,Geelong,"Oct 16, 2022",T20I # 1825
2,Scotland,West Indies,Scotland,42 runs,Hobart,"Oct 17, 2022",T20I # 1826
3,Ireland,Zimbabwe,Zimbabwe,31 runs,Hobart,"Oct 17, 2022",T20I # 1828
4,Namibia,Netherlands,Netherlands,5 wickets,Geelong,"Oct 18, 2022",T20I # 1830


In [18]:
df_match.to_csv('C:/Users/SKS/Desktop/IPL project/match_info.csv',index=False)

**2. Scraping Bowling Summary**

Batting data is gathered from individual scorecard of matches

In [4]:
data  = requests.get(url).text 
soup = BeautifulSoup(data,"html.parser") 

In [5]:
tables=soup.find_all('table')

In [6]:
for index,table in enumerate(tables):
    if ("Match results" in str(table)):
        table_index = index
        print(table_index)

0


**2.1 Storing all the scorecard links in a list**

In [7]:
links_list=[]
for link in tables[table_index].find_all('a'):
    links_list.append(link.get('href'))


In [8]:
result=[i for i in links_list if "match" in i]
final_links=['https://stats.espncricinfo.com'+s for s in result]


In [9]:
df_links= pd.DataFrame({'links':final_links,'match_id':df_match['Scorecard']})


In [10]:
df_links.drop(labels=[20,24,25],axis=0,inplace=True)


In [11]:
df_links.reset_index(inplace=True)
df_links.drop(columns=['index'],inplace=True)
df_links.head()

,links,match_id
0,https://stats.espncricinfo.com/ci/engine/match...,T20I # 1823
1,https://stats.espncricinfo.com/ci/engine/match...,T20I # 1825
2,https://stats.espncricinfo.com/ci/engine/match...,T20I # 1826
3,https://stats.espncricinfo.com/ci/engine/match...,T20I # 1828
4,https://stats.espncricinfo.com/ci/engine/match...,T20I # 1830


**2.2 Code for scraping bowling data**

In [208]:
df_bowling=pd.DataFrame()
for count,i in enumerate(df_links['links']):
    df_bowling1=pd.read_html(i)[1]
    df_bowling2=pd.read_html(i)[3]
    df_bowl_full=pd.concat([df_bowling1,df_bowling2],ignore_index=True)
    df_bowl_full=df_bowl_full[df_bowl_full['BOWLING'].apply(lambda x:  len(x)<30 )]
    df_bowl_full.reset_index(drop=True)
    df_bowl_full['match_id']=df_links['match_id'][count]
    df_bowling=pd.concat([df_bowling,df_bowl_full],ignore_index=True)
    
df_bowling  

,BOWLING,O,M,R,W,ECON,0s,4s,6s,WD,NB,match_id
0,Maheesh Theekshana,4,0,23,1,5.75,7,0,0,2,0,T20I # 1823
1,Dushmantha Chameera,4,0,39,1,9.75,6,3,1,2,0,T20I # 1823
2,Pramod Madushan,4,0,37,2,9.25,6,3,1,0,0,T20I # 1823
3,Chamika Karunaratne,4,0,36,1,9.00,7,3,1,1,0,T20I # 1823
4,Wanindu Hasaranga de Silva,4,0,27,1,6.75,8,1,1,0,0,T20I # 1823
...,...,...,...,...,...,...,...,...,...,...,...,...
495,Naseem Shah,4,0,30,0,7.50,15,3,1,1,0,T20I # 1879
496,Haris Rauf,4,0,23,2,5.75,13,3,0,1,0,T20I # 1879
497,Shadab Khan,4,0,20,1,5.00,10,1,0,0,0,T20I # 1879
498,Mohammad Wasim,4,0,38,1,9.50,5,5,0,2,0,T20I # 1879


**3. Code for scraping batting data**

In [239]:
df_batting=pd.DataFrame()
for count,i in enumerate(df_links['links']):
    df_batting1=pd.read_html(i)[0]
    df_batting2=pd.read_html(i)[2]
    df_batting1=df_batting1.iloc[:df_batting1[df_batting1.BATTING=='Extras'].index[0],:-2].dropna()
    df_batting1.reset_index(inplace=True,drop=True)
    df_batting1['battingPos']=df_batting1.index+1
    df_batting2=df_batting2.iloc[:df_batting2[df_batting2.BATTING=='Extras'].index[0],:-2].dropna()
    df_batting2.reset_index(inplace=True,drop=True)
    df_batting2['battingPos']=df_batting2.index+1
    df_bat_full=pd.concat([df_batting1,df_batting2])
    df_bat_full['match_id']=df_links['match_id'][count]
    df_batting=pd.concat([df_batting,df_bat_full])
df_batting
    
   

,BATTING,Unnamed: 1,R,B,M,4s,6s,SR,battingPos,match_id
0,Michael van Lingen,c Pramod Madushan b Chameera,3,6,7,0,0,50.00,1,T20I # 1823
1,Divan la Cock,c Shanaka b Pramod Madushan,9,9,15,1,0,100.00,2,T20I # 1823
2,Jan Nicol Loftie-Eaton,c †Mendis b Karunaratne,20,12,18,1,2,166.66,3,T20I # 1823
3,Stephan Baard,c DM de Silva b Pramod Madushan,26,24,49,2,0,108.33,4,T20I # 1823
4,Gerhard Erasmus (c),c Gunathilaka b PWH de Silva,20,24,30,0,0,83.33,5,T20I # 1823
...,...,...,...,...,...,...,...,...,...,...
2,Phil Salt,c Iftikhar Ahmed b Haris Rauf,10,9,16,2,0,111.11,3,T20I # 1879
3,Ben Stokes,not out,52,49,81,5,1,106.12,4,T20I # 1879
4,Harry Brook,c Shaheen Shah Afridi b Shadab Khan,20,23,36,1,0,86.95,5,T20I # 1879
5,Moeen Ali,b Mohammad Wasim,19,13,30,3,0,146.15,6,T20I # 1879


In [240]:
df_batting.to_csv('batting_uncleaned.csv',index=False)

**4. Information about players regarding their team, playing role, batting or bowling style and description about them which would be useful in determing the role the selected players would fit in for the dream team**

In [246]:

player_data  = requests.get(final_links[0]).text 
player_soup = BeautifulSoup(player_data,"html.parser")

In [267]:
match_tables=player_soup.find_all('table')

**4.1 Creating a list of links with info of players**

In [276]:
list_2=[]
for j in final_links:
    player_data  = requests.get(j).text 
    player_soup = BeautifulSoup(player_data,"html.parser")
    match_tables=player_soup.find_all('table')
    for i in range(4):
        for link in match_tables[i].find_all('a',href=True):  # in html anchor/link is represented by the tag <a>

            list_2.append(link.get('href'))
    
res=[i for i in list_2 if "cricketers" in i]
res

['/cricketers/michael-van-lingen-833777',
 '/cricketers/divan-la-cock-1178065',
 '/cricketers/jan-nicol-loftie-eaton-1071473',
 '/cricketers/stephan-baard-437927',
 '/cricketers/gerhard-erasmus-519070',
 '/cricketers/jan-frylinck-482478',
 '/cricketers/david-wiese-221140',
 '/cricketers/jj-smit-553821',
 '/cricketers/zane-green-596664',
 '/cricketers/bernard-scholtz-330223',
 '/cricketers/ben-shikongo-1069820',
 '/cricketers/maheesh-theekshana-1138316',
 '/cricketers/dushmantha-chameera-552152',
 '/cricketers/pramod-madushan-826689',
 '/cricketers/chamika-karunaratne-623695',
 '/cricketers/wanindu-hasaranga-de-silva-784379',
 '/cricketers/pathum-nissanka-1028655',
 '/cricketers/kusal-mendis-629074',
 '/cricketers/dhananjaya-de-silva-465793',
 '/cricketers/danushka-gunathilaka-345821',
 '/cricketers/bhanuka-rajapaksa-342619',
 '/cricketers/dasun-shanaka-437316',
 '/cricketers/wanindu-hasaranga-de-silva-784379',
 '/cricketers/chamika-karunaratne-623695',
 '/cricketers/pramod-madushan-826

Extracting Unique links

In [280]:
player_links=set(res)

In [283]:
final_player_links=['https://stats.espncricinfo.com'+s for s in player_links]

In [317]:
player_data  = requests.get(final_player_links[20]).text 
player_soup = BeautifulSoup(player_data,"html.parser")
final_player_links[20]

'https://stats.espncricinfo.com/cricketers/glenn-maxwell-325026'

**4.2 Code for scraping player information**

In [482]:
df_player_info=pd.DataFrame(columns=['name','team','battingStyle','bowlingStyle','playingRole','description'])
for index,i in enumerate(final_player_links):
    player_data  = requests.get(i).text 
    player_soup = BeautifulSoup(player_data,"html.parser")
    test_2=player_soup.find_all(class_="ds-text-tight-m ds-font-regular ds-uppercase ds-text-typo-mid3")
    flag=0
    for i in test_2 :
        if "Other" in i:
            flag=1
    if flag==1:
        print('yes')
        continue;
    match_tables=player_soup.find(class_='ds-cursor-pointer ds-inline-flex ds-items-start ds-leading-none')
    Team=match_tables.text
    match_tables=player_soup.find('h1')
    Name=match_tables.text
    match_tables=player_soup.find_all('h5')
    
    test=player_soup.find_all(class_="ds-text-tight-m ds-font-regular ds-uppercase ds-text-typo-mid3")
    
    if 'Also Known As' in test[3]:
        BattingStyle=match_tables[4].text
        if 'Bowling Style'in  test[5]:
            BowlingStyle=match_tables[5].text
        else:
            BowlingStyle='Bat'
        PlayingRole=match_tables[6].text
    else:
        BattingStyle=match_tables[3].text 
        if 'Bowling Style'in  test[4]:
            BowlingStyle=match_tables[4].text
        else:
            BowlingStyle='Bat'
        PlayingRole=match_tables[5].text
    
        
        
    
    
    match_tables=player_soup.find(class_='ci-player-bio-content')
    if match_tables==None:
        Description=''
    else:
        Description=match_tables.p.text
        

    df_player_info=df_player_info.append({'name':Name,'team':Team,'battingStyle':BattingStyle,'bowlingStyle':BowlingStyle,'playingRole':PlayingRole,'description':Description}, ignore_index=True)
    print(index)
df_player_info

C:\Users\SKS\AppData\Local\Temp\ipykernel_964\433721850.py:47: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_player_info=df_player_info.append({'name':Name,'team':Team,'battingStyle':BattingStyle,'bowlingStyle':BowlingStyle,'playingRole':PlayingRole,'description':Description}, ignore_index=True)


0


C:\Users\SKS\AppData\Local\Temp\ipykernel_964\433721850.py:47: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_player_info=df_player_info.append({'name':Name,'team':Team,'battingStyle':BattingStyle,'bowlingStyle':BowlingStyle,'playingRole':PlayingRole,'description':Description}, ignore_index=True)


1


C:\Users\SKS\AppData\Local\Temp\ipykernel_964\433721850.py:47: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_player_info=df_player_info.append({'name':Name,'team':Team,'battingStyle':BattingStyle,'bowlingStyle':BowlingStyle,'playingRole':PlayingRole,'description':Description}, ignore_index=True)


...

230


C:\Users\SKS\AppData\Local\Temp\ipykernel_964\433721850.py:47: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_player_info=df_player_info.append({'name':Name,'team':Team,'battingStyle':BattingStyle,'bowlingStyle':BowlingStyle,'playingRole':PlayingRole,'description':Description}, ignore_index=True)


231
232


C:\Users\SKS\AppData\Local\Temp\ipykernel_964\433721850.py:47: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_player_info=df_player_info.append({'name':Name,'team':Team,'battingStyle':BattingStyle,'bowlingStyle':BowlingStyle,'playingRole':PlayingRole,'description':Description}, ignore_index=True)


,name,team,battingStyle,bowlingStyle,playingRole,description
0,Michael Bracewell,New Zealand,Left hand Bat,Right arm Offbreak,Wicketkeeper,The latest member of the Bracewell dynasty to ...
1,Roelof van der Merwe,Netherlands,Right hand Bat,Slow Left arm Orthodox,Allrounder,Tidy left-arm spinner and hard-hitting batter ...
2,Akeal Hosein,West Indies,Left hand Bat,Slow Left arm Orthodox,Bowler,Akeal Hosein was handed a chance to impress fo...
3,Barry McCarthy,Ireland,Right hand Bat,Right arm Fast medium,Bowler,Barry McCarthy is an Ireland seam bowler whose...
4,Harry Tector,Ireland,Right hand Bat,Right arm Offbreak,Middle order Batter,Born into a cricketing family - his brothers J...
...,...,...,...,...,...,...
220,Aiden Markram,South Africa,Right hand Bat,Right arm Offbreak,Opening Batter,The first South Africa captain to win a World ...
221,Regis Chakabva,Zimbabwe,Right hand Bat,Right arm Offbreak,Wicketkeeper,"A doughty wicketkeeper-batsman, Regis Chakabva..."
222,Rilee Rossouw,South Africa,Left hand Bat,Right arm Offbreak,Top order Batter,A stylish top-order player with an elegant and...
223,Haider Ali,Pakistan,Right hand Bat,Bat,Batting & Fielding,


In [483]:
df_player_info.to_csv('C:/Users/SKS/Desktop/IPL project/player_info_new2.csv',index=False)